# Descrição do projeto

## Variáveis globais
- `API_BASE_URL` (str): URL base da API usada para CRUD de jogadores.
- `GEMINI_API_KEY` (str): chave da API do Google Gemini (usada para configurar `genai`).
- `model` (GenerativeModel): instância do cliente Gemini configurada para gerar recomendações.

---

## Célula 1 — create_test_players
- Importa `requests`.
- Define `API_BASE_URL`.
- `create_test_players()`:
    - Monta uma lista de 3 objetos de jogador (id, psn_id, name, favorite_genre, subscription, console, notifications).
    - Para cada jogador faz `POST` em `{API_BASE_URL}/players`.
    - Imprime status por jogador: sucesso (201), já existe (400) ou erro (outros códigos).
- Executa `create_test_players()` quando o módulo é executado como script (`if __name__ == "__main__":`).
- Efeito colateral: cria/insere registros na API remota.
- Observações:
    - O intuito é popular a API com dados de teste para o pipeline ETL.



In [ ]:
import requests

# 1. Ajuste para a URL da sua API no Render
API_BASE_URL = "https://bootcamp-ciencia-dados-com-python.onrender.com"

def create_test_players():
    players = [
        {
            "id": 101,
            "psn_id": "Dvic_Gamer",
            "name": "Dvic",
            "favorite_genre": "RPG / Soulslike",
            "subscription": {
                "plan": "Plus Deluxe",
                "status": "Active",
                "expiry_date": "2025-12-31",
                "level": 350
            },
            "console": {
                "model": "PlayStation 5 Slim",
                "serial_number": "ABC-123456",
                "storage_used_gb": 720.5
            },
            "notifications": []
        },
        {
            "id": 102,
            "psn_id": "Kratos_BR",
            "name": "Gabriel",
            "favorite_genre": "Action / Adventure",
            "subscription": {
                "plan": "Plus Extra",
                "status": "Active",
                "expiry_date": "2024-06-15",
                "level": 120
            },
            "console": {
                "model": "PlayStation 5",
                "serial_number": "XYZ-987654",
                "storage_used_gb": 400.0
            },
            "notifications": []
        },
        {
            "id": 103,
            "psn_id": "Spider_Gwen",
            "name": "Mariana",
            "favorite_genre": "Open World",
            "subscription": {
                "plan": "Plus Essential",
                "status": "Active",
                "expiry_date": "2025-01-20",
                "level": 45
            },
            "console": {
                "model": "PlayStation 4 Pro",
                "serial_number": "PRO-555666",
                "storage_used_gb": 850.0
            },
            "notifications": []
        }
    ]

    print(f"Conectando à API em: {API_BASE_URL}")
    for player in players:
        response = requests.post(f"{API_BASE_URL}/players", json=player)
        if response.status_code == 201:
            print(f"Jogador {player['psn_id']} cadastrado com sucesso!")
        elif response.status_code == 400:
            print(f"Jogador {player['psn_id']} já existe na base.")
        else:
            print(f"Erro ao cadastrar {player['psn_id']}: {response.status_code}")

if __name__ == "__main__":
    create_test_players()

Conectando à API em: https://bootcamp-ciencia-dados-com-python.onrender.com
Jogador Dvic_Gamer cadastrado com sucesso!
Jogador Kratos_BR cadastrado com sucesso!
Jogador Spider_Gwen cadastrado com sucesso!


---

## Célula 2 — Pipeline ETL de recomendações (Run Pipeline)
- Importa `requests`, `pandas`, `google.generativeai as genai`, `time`.
- Configura `genai` com `GEMINI_API_KEY` e instancia `model` (`gemini-2.5-flash`).
- Funções:
    - `extract_user_ids(file_path)`: lê CSV local, retorna lista da coluna `player_id`; trata erros e retorna lista vazia se falhar.
    - `get_player_data(player_id)`: faz `GET` em `{API_BASE_URL}/players/{player_id}`, retorna JSON se status 200, senão `None`.
    - `transform_generate_recommendation(player)`: monta prompt com nome/gênero/plano e chama `model.generate_content(prompt)` para gerar uma mensagem curta (menor ou igual a 140 caracteres); em caso de erro ou resposta vazia retorna mensagem fallback.
    - `load_update_player(player)`: faz `PUT` em `{API_BASE_URL}/players/{id}` com o JSON do jogador; retorna `True` se status 200.
    - `run_pipeline()`: orquestra o ETL — lê IDs de `jogadores_id.csv`, para cada ID extrai dados, gera recomendação com Gemini, anexa notificação em `player['notifications']`, faz `PUT` para atualizar na API, imprime logs e aguarda 2s entre requisições (respeito a limites).
- Requisitos/observações:
    - Depende de arquivo `jogadores_id.csv` com coluna `player_id`.
    - Requer conectividade com a API em `API_BASE_URL` e chave válida `GEMINI_API_KEY`.
    - Produz chamadas de rede e modifica dados na API remota.
    
- `IMPORTANTE`: 
    - A CHAVE DA API PRECISA SER PASSADA PARA FUNCIONAR, DO CONTRÁRIO, POR MOTIVOS ÓBVIOS, RESULTARÁ EM ERRO.
    - Favor acessar https://aistudio.google.com/app/api-keys para copiar uma chave válida, para assim poderem avaliar a funcionalidade do pipeline.

In [ ]:
import requests
import pandas as pd
import google.generativeai as genai
import time

# --- CONFIGURAÇÕES ---
# 1. URL da sua API no Render (Certifique-se de que é a URL do Web Service)
API_BASE_URL = "https://bootcamp-ciencia-dados-com-python.onrender.com"

# 2. API Key do Google AI Studio (Gemini) Substitua pela sua chave real
GEMINI_API_KEY = "SUA_CHAVE_AQUI"

# Configuração do Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Ajuste do modelo para a versão estável mais recente
model = genai.GenerativeModel('gemini-2.5-flash')

def extract_user_ids(file_path):
    """Extração: Lê os IDs dos jogadores de um arquivo CSV local"""
    try:
        df = pd.read_csv(file_path)
        return df['player_id'].tolist()
    except Exception as e:
        print(f"Erro ao ler arquivo de IDs: {e}")
        return []

def get_player_data(player_id):
    """Extração: Busca os dados do jogador na API através do ID"""
    response = requests.get(f"{API_BASE_URL}/players/{player_id}")
    return response.json() if response.status_code == 200 else None

def transform_generate_recommendation(player):
    """Transformação: O Gemini analisa os gêneros favoritos e sugere um jogo"""
    name = player['name']
    genre = player['favorite_genre']
    plan = player['subscription']['plan']

    # Prompt focado em recomendações personalizadas por gênero
    prompt = (
        f"Você é um especialista em marketing da PlayStation. "
        f"O jogador {name} ama o gênero {genre} e possui o plano de assinatura {plan}. "
        f"Crie uma mensagem curta e empolgante recomendando um jogo de {genre} "
        f"que esteja disponível para o plano dele. "
        f"A mensagem deve ser direta e ter no máximo 140 caracteres."
    )

    try:
        # Chamada à API do Gemini para gerar a recomendação
        response = model.generate_content(prompt)
        # Verifica se a resposta foi gerada com sucesso
        if response and response.text:
            return response.text.strip()
        else:
            return f"Fala {name}! Temos novos jogos de {genre} esperando por você na Plus!"
    except Exception as e:
        print(f"Erro ao gerar recomendação para {name}: {e}")
        return f"Fala {name}! Temos novos jogos de {genre} esperando por você na Plus!"

def load_update_player(player):
    """Carga: Atualiza o perfil do jogador na API com a nova notificação"""
    response = requests.put(f"{API_BASE_URL}/players/{player['id']}", json=player)
    return response.status_code == 200

# --- EXECUÇÃO DO FLUXO ETL ---

def run_pipeline():
    print("Iniciando Pipeline de Marketing PlayStation...")
    
    # 1. EXTRACT
    player_ids = extract_user_ids('jogadores_id.csv')
    if not player_ids:
        print("Pipeline interrompido: Nenhum ID para processar.")
        return

    for p_id in player_ids:
        print(f"\nExtraindo dados do Jogador ID: {p_id}...")
        player = get_player_data(p_id)
        
        if not player:
            print(f"Jogador {p_id} não encontrado na API.")
            continue
        
        # 2. TRANSFORM
        print(f"Gerando recomendação para {player['name']}...")
        recommendation_msg = transform_generate_recommendation(player)
        
        # Adiciona a mensagem gerada pela IA na lista de notificações
        new_notification = {
            "type": "AI Recommendation",
            "message": recommendation_msg
        }
        player['notifications'].append(new_notification)
        
        # 3. LOAD
        print(f"Enviando atualização para a API...")
        success = load_update_player(player)
        
        if success:
            print(f"Sucesso: {player['name']} recebeu a mensagem: '{recommendation_msg}'")
        else:
            print(f"Erro ao carregar dados de {player['name']} na API.")
            
        # Delay para respeitar os limites da API gratuita
        time.sleep(2)

if __name__ == "__main__":
    run_pipeline()

Iniciando Pipeline de Marketing PlayStation...

Extraindo dados do Jogador ID: 101...
Jogador 101 não encontrado na API.

Extraindo dados do Jogador ID: 102...
Jogador 102 não encontrado na API.

Extraindo dados do Jogador ID: 103...
Jogador 103 não encontrado na API.
